In [2]:
import os
from itertools import product
from collections import namedtuple
import time
from typing import Tuple
import random
import math

from tabulate import tabulate
from tqdm.notebook import tqdm
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import patches
import numpy as np
import pandas as pd

# Подключаем локальные пакеты
from pyqumo.random import Distribution, Exponential, HyperExponential, Erlang
from pyqumo.cqumo.sim import simulate_tandem


from pyqumo.fitting.johnson89 import fit_mern2
from pyqumo.stats import get_cv, get_skewness, get_noncentral_m2, get_noncentral_m3
from pyqumo.random import HyperErlang

# Поключаем пакеты для ML

# from sklearn.metrics import r2_score, mean_squared_error, precision_score, recall_score, f1_score
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.tree import DecisionTreeClassifier
# from sklearn import tree
# from sklearn.ensemble import GradientBoostingRegressor
# from sklearn.model_selection import train_test_split
# from sklearn.model_selection import GridSearchCV
# from sklearn.neural_network import MLPClassifier
# from sklearn.linear_model import LinearRegression, LogisticRegression
# from catboost import CatBoostClassifier
# import tensorflow as tf
# from tensorflow import keras
# from tensorflow import keras
# from tensorflow.python.keras import engine
# from tensorflow.keras import layers
# import seaborn as sns
# from scipy import stats

In [145]:
def fit_ph(avg: float, std: float) -> Distribution:
    """
    Возвращает PH-распределение по стандартному отклонению и среднему значению.
    """
    cv = std / avg
    if (cv == 1) or math.isclose(avg, std, rel_tol=0.01):
        return Exponential(1/avg)
    if cv > 1:
        return HyperExponential.fit(avg, std)
    return Erlang.fit(avg, std)

In [5]:
 COLUMNS = ['sta_where_packets_arrive',
            'simulation_type',
            'num_generated_packets',
            'mode',
            'num_stations',
            'queue_capacity',
            'mean_arrival_time',
            'std_arrival_time',
            'bitrate',
            'mean_packet_size',
            'std_mean_packet_size']

In [152]:
def simulate(
    df: pd.DataFrame, *,
    sta_where_packets_arrive: any,
    simulation_type: str,
    num_generated_packets: int,
    mode: str,
    num_stations: int,
    queue_capacity: int,
    mean_arrival_time: float,
    std_arrival_time: float,
    bitrate: float,
    mean_packet_size: float,
    std_mean_packet_size:float) -> pd.DataFrame:

    arr = fit_ph(mean_arrival_time, 
                 std_arrival_time)

    srv = fit_ph(mean_packet_size/bitrate, 
                 std_mean_packet_size/bitrate)

    ret = simulate_tandem([arr] * int(num_stations), 
                    [srv] * int(num_stations), 
                    queue_capacity=queue_capacity, 
                    fixed_service=simulation_type=="FIXED_PACKET_SIZE", 
                    max_packets=num_generated_packets)
    row_data = {
        'sta_where_packets_arrive': sta_where_packets_arrive,
        'simulation_type': simulation_type,
        'num_generated_packets': num_generated_packets,
        'mode': mode,
        'num_stations': num_stations,
        'queue_capacity': queue_capacity,
        'mean_arrival_time': mean_arrival_time,
        'std_arrival_time': std_arrival_time,
        'bitrate': bitrate,
        'mean_packet_size': mean_packet_size,
        'std_mean_packet_size': std_mean_packet_size,
        'mean_service_time': srv.moment(1),
        'e2e_delay': ret.delivery_delays[0].avg,
        'e2e_delay_std': ret.delivery_delays[0].std,
        'delivery_prob': ret.delivery_pro200b[0],
        'rho': srv.moment(1) / srv.moment(2),
    }

    df = df.append(row_data, ignore_index=True)

    return df

In [96]:
def prepare_inputs(sample_num: int, data: dict) -> dict:
    swpa = np.zeros(sample_num)
    simtype = [None] * sample_num
    numgenp = np.zeros(sample_num)
    mode = [None] * sample_num
    num_sta = np.zeros(sample_num)
    bitrate = np.zeros(sample_num)
    qcapacity = np.zeros(sample_num)
    mean_a = np.zeros(sample_num)
    std_a = np.zeros(sample_num)
    mean_size = np.zeros(sample_num)
    std_size = np.zeros(sample_num)

    for i in range(sample_num):
        swpa[i] = data['STA_WHERE_PACKETS_ARRIVE']
        simtype[i] = data['SIMULATION_TYPE']
        numgenp[i] = data['NUM_GENERATED_PACKETS']
        mode[i] = data['MODE']
        num_sta[i] = np.random.randint(data['NUM_STATIONS'][0],
                                       data['NUM_STATIONS'][-1] + 1)
        bitrate[i] = round(np.random.uniform(data['BITRATE'][0],
                                             data['BITRATE'][-1] + 1), 3)
        qcapacity[i] = np.random.randint(data['QUEUE_CAPACITY'][0],
                                         data['QUEUE_CAPACITY'][-1] + 1)
        mean_a[i] = round(np.random.uniform(data['MEAN_ARRIVAL_TIME'][0],
                                            data['MEAN_ARRIVAL_TIME'][-1]), 3)
        std_a[i] = round(np.random.uniform(data['STD_ARRIVAL_TIME'][0],
                                           data['STD_ARRIVAL_TIME'][-1]), 3)
        mean_size[i] = round(np.random.uniform(data['MEAN_PACKET_SIZE'][0],
                                               data['MEAN_PACKET_SIZE'][-1]), 3)
        std_size[i] = round(np.random.uniform(data['STD_PACKET_SIZE'][0],
                                              data['STD_PACKET_SIZE'][-1]), 3)
    return dict(zip(COLUMNS, [swpa, simtype, numgenp, mode, num_sta,
                              qcapacity, mean_a, std_a,  bitrate, 
                              mean_size, std_size]))

In [207]:
SAMPLE_NUM = 100
# DATA = {
#   "STA_WHERE_PACKETS_ARRIVE": -1,
#   "SIMULATION_TYPE" : "FIXED_PACKET_SIZE",
#   "NUM_GENERATED_PACKETS": 100000,
#   "MODE": "UPLOAD",
#   "NUM_STATIONS" : [1, 20],
#   "BITRATE": [500, 600],
#   "QUEUE_CAPACITY": [5, 10],
#   "MEAN_ARRIVAL_TIME": [50, 100],
#   "STD_ARRIVAL_TIME": [50, 60],
#   "MEAN_PACKET_SIZE": [500, 640],
#   "STD_PACKET_SIZE": [500, 520]
# }
DATA = {
  "STA_WHERE_PACKETS_ARRIVE": -1,
  "SIMULATION_TYPE" : "FIXED_PACKET_SIZE",
  "NUM_GENERATED_PACKETS": 100000,
  "MODE": "UPLOAD",
  "NUM_STATIONS" : [1, 20],
  "BITRATE": [300, 400],
  "QUEUE_CAPACITY": [5, 10],
  "MEAN_ARRIVAL_TIME": [20, 50],
  "STD_ARRIVAL_TIME": [10, 50],
  "MEAN_PACKET_SIZE": [1500, 4000],
  "STD_PACKET_SIZE": [1000, 4000]
}

input_data = prepare_inputs(SAMPLE_NUM, DATA)

In [208]:
sim_data = pd.DataFrame(columns=COLUMNS)
for i in tqdm(range(SAMPLE_NUM)):
        sim_data = simulate(
            sim_data,
            sta_where_packets_arrive = input_data['sta_where_packets_arrive'][i],
            simulation_type = input_data['simulation_type'][i],
            num_generated_packets = int(input_data['num_generated_packets'][i]),
            mode = input_data['mode'][i],
            num_stations = int(input_data['num_stations'][i]),
            queue_capacity = int(input_data['queue_capacity'][i]),
            mean_arrival_time = input_data['mean_arrival_time'][i],
            std_arrival_time = input_data['std_arrival_time'][i],
            bitrate = input_data['bitrate'][i],
            mean_packet_size = input_data['mean_packet_size'][i],
            std_mean_packet_size = input_data['std_mean_packet_size'][i])

sim_data.to_csv('data/tandem_network_crosstraffic.csv')

  0%|          | 0/100 [00:00<?, ?it/s]